In [1]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import timeit
import datetime
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nile.api.v1 import clusters
from nile.api.v1 import aggregators as na
from qb2.api.v1 import filters as qf
from nile.api.v1 import filters as nf
from nile.api.v1 import extractors as ne
import ast
from collections import defaultdict
from keras.preprocessing import sequence
import time
from gensim.models import word2vec
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers.advanced_activations import PReLU
import matplotlib
import pickle
import gc
from keras.models import load_model
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation
from __future__ import print_function
from time import time
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,HashingVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from tqdm import tqdm
import pymorphy2
import nltk

/usr/local/lib/python2.7/dist-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:28: UserWarning: [Errno 2] No such file or directory.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
/usr/lib/python2.7/dist-packages/joblib/_multiprocessing_helpers.py:29: UserWarning: [Errno 2] No such file or directory.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
Using TensorFlow backend.


In [2]:
n_samples = 2000
n_features = 30000
n_components = 10
n_top_words = 40
morph = pymorphy2.MorphAnalyzer()
stopWords = stopwords.words('russian')
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words=stopWords)#,input="file")

Extracting tf features for LDA...


In [3]:
features_to_exclude = pickle.load(open('features_not_relevant', 'rb'))
features_to_include = pickle.load(open('features_relevant', 'rb'))

### Лемматизация и стоп символы

In [4]:
from string import punctuation
from nltk.tokenize import WhitespaceTokenizer
exclude = set(punctuation + '0123456789'+u'–—'+u'«»')
import util_david

In [5]:
lemma_dict = {}
def lemmatizer(word):
    if word in lemma_dict:
        return lemma_dict[word]
    else:
        normal_form = morph.parse(word)[0].normal_form
        lemma_dict[word] = normal_form
        return normal_form

### Загрузка данных  - dataset1.csv =  это переименованный dataset1_ver2.csv

In [6]:
x1  = pd.read_csv('data/dataset1.csv') #
x2  = pd.read_csv('data/dataset2.csv')
x3  = pd.read_csv('data/views.csv',sep = ';')
x4  = pd.read_csv('data/compare_v3.csv',sep = ';')
dist_categories_id = list(x1.CATEGORY_ID.unique())
dist_categories_names = list(x1.CATEGORY_NAME.unique())
mappings = x1[["CATEGORY_ID",'CATEGORY_NAME']].drop_duplicates()
mappings['Number_of_reviews'] = 0

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()

In [9]:
reserv = 'iPhone 6'
name = raw_input() or reserv
c = 0
iphone_6_review = []
for i in xrange(x1.shape[0]):
    try:
        if name in x1.NAME.values[i]:
            c+=1
            iphone_6_review.append(i)
            #break
    except:
        continue
print(c)
iphone6 = x1.ix[iphone_6_review,:]

res_iphone_6 = []
for person in tqdm(iphone6.TEXT.values,miniters=10000):
    new_string = ""
    for sentence in person.decode('utf-8').split():
        for char in exclude:
            sentence = sentence.replace(char,"")
        new_string =new_string +' '+ sentence
    new_string_2 = ""
    for word in new_string.split():
        new_string_2 = new_string_2 + " "+ lemmatizer(word)
        

    res_iphone_6.append(new_string_2)
gc.collect()

tf = tf_vectorizer.fit_transform(res_iphone_6)
y = iphone6.RATING
model.fit(tf,y)

fi_ip = model.feature_importances_
tf_feature_names = tf_vectorizer.get_feature_names()
fi_ip = pd.DataFrame(fi_ip)
fi_ip['word'] = tf_feature_names
fi_ip.rename(columns={0:'weight'},inplace=True)
fi_ip.sort(columns='weight',ascending=False,inplace=True)
fi_ip = fi_ip[fi_ip.weight>0]
fi_ip['exclude'] = 0

for n,word in enumerate(fi_ip.word.values):
    if word in features_to_exclude:
        fi_ip.exclude.values[n] = 1
fi_ip_adjusted = fi_ip[fi_ip.exclude==0]

fi_ip_adjusted.weight = fi_ip_adjusted.weight/np.sum(fi_ip_adjusted.weight)
fi_ip_adjusted.weight = np.float16(fi_ip_adjusted.weight)

print ('not relevant excluded')
print(fi_ip_adjusted[0:30])
print(100*'-')
tf = tf_vectorizer.fit_transform(res_iphone_6)
y = iphone6.RATING
model.fit(tf,y)

fi_ip = model.feature_importances_
tf_feature_names = tf_vectorizer.get_feature_names()
fi_ip = pd.DataFrame(fi_ip)
fi_ip['word'] = tf_feature_names
fi_ip.rename(columns={0:'weight'},inplace=True)
fi_ip.sort(columns='weight',ascending=False,inplace=True)
fi_ip = fi_ip[fi_ip.weight>0]
fi_ip['exclude'] = 1

for n,word in enumerate(fi_ip.word.values):
    if word in features_to_include:
        fi_ip.exclude.values[n] = 0
fi_ip_adjusted = fi_ip[fi_ip.exclude==0]

fi_ip_adjusted.weight = fi_ip_adjusted.weight/np.sum(fi_ip_adjusted.weight)
fi_ip_adjusted.weight = np.float16(fi_ip_adjusted.weight)
print ('only relevant included')

print(fi_ip_adjusted[0:30])

iPhone 6


  0%|          | 0/780 [00:00<?, ?it/s]

780


100%|██████████| 780/780 [00:03<00:00, 241.91it/s]


not relevant excluded
        weight          word  exclude
1942  0.142212         тихий        0
1105  0.084534    олеофобный        0
382   0.080078        деньга        0
1632  0.069275        ремонт        0
1563  0.044037          пыль        0
845   0.036316      мерцание        0
73    0.032166    аккуратный        0
1309  0.030060     подсветка        0
1255  0.028671       пластик        0
1947  0.028534         толща        0
821   0.025864       матовый        0
654   0.024490        камера        0
1668  0.022629       самсунг        0
2127  0.021927          цена        0
1206  0.017822         ощупь        0
725   0.016220        корпус        0
820   0.015686      материал        0
556   0.015350        звонок        0
1868  0.014801        стекло        0
1732  0.014366       скачать        0
1663  0.014351      садиться        0
664   0.013771  качественный        0
532   0.013077         заряд        0
1594  0.011810        размер        0
1267  0.011246       площадь